In [1]:
# Configurar el path para importar los módulos
import sys
import os

# Añadir el directorio raíz del proyecto al path
root_dir = os.chdir(os.path.abspath(os.path.join(os.getcwd(), '..')))
if root_dir not in sys.path:
    sys.path.append(root_dir)

In [2]:
# Importar las funciones necesarias
from db.conexion_db import crear_conexion, cerrar_conexion
from db.querys_db import insert_data, consult_data,procedure_data
from abastecimiento import reservas_avisorden_ordenes
from cruce_inventario import cruce_inventarios
import pandas as pd
import numpy as np

In [3]:
tbl_pedidos_pendientes = 'tbl_pedidospendientes_me2n'
plan_pedidospendientes= 'plan_pedidospendientes'
rpt_estado_compras_pendientes = 'rpt_estado_compras_pendientes'
plan_pedidospendientes_resumen = 'plan_pedidospendientes_resumen'
plan_trazabilidad = "plan_trazabilidad_compras"

cabezeras_estado_compra = ['id','fechacarga','doc_compra','pos','contr','pos1','orgc','un','rep','cl_documento','tipo_documento','grupo_documento','Hist_ped','Fecha_doc','Nombre_proveedor','Centro_suministrador','Material','txt_breve','Gpo_articulo','I_borrado',
                        'T_posicion','T_imp','Centro','Alm','Cantidad','Por_entr_Ctd','UMP','Precio_neto','Valor_neto','Val_pend_total','Val_pend','Por_entr_valor','Moneda','c_base','ctd_reparto','Fecha_entrega','Hora','Fech_entr_estad','Ctd_ante','Ctd_ent',
                        'Salida','Ctd_entr','Pedido','Pos_pedido','Ind_creacion','Ctd_posicion','Num_necesidad','Fech_solicitud','Fe_pedido','Fe_entr_pre','Fe_doc_en','Estado','Liberador_1','Liberador_2','Liberador_3','Liberador_4','Creador_doc_comp']



In [4]:
connection = crear_conexion()
consult_data(connection,f"truncate table {plan_pedidospendientes}")
consult_data(connection,f"truncate table {rpt_estado_compras_pendientes}")
consult_data(connection,f"truncate table {plan_pedidospendientes_resumen}")
insert_plan_pedidospendiente = f"insert into {plan_pedidospendientes} select * from {tbl_pedidos_pendientes} where fechacarga = (select max(fechacarga) from {tbl_pedidos_pendientes}) "
consult_data(connection,insert_plan_pedidospendiente)

procedure_data(connection,'abastece_plan_trazabilidad')
cerrar_conexion(connection)

Conexion Exitosa
El procedimiento se ah ejecutado correctamente
conexion cerrada


In [5]:
connection = crear_conexion()
pedidospendiente = consult_data(connection,f"select * from {plan_pedidospendientes}")
trazabilidad = consult_data(connection,f"select Fech_solicitud,Fe_pedido,Fe_entr_pre,Fe_doc_en,Estado,Liberador_1,Liberador_2,Liberador_3,Liberador_4,Creador_doc_comp,Doc_compra,Pos_doc_cmp from {plan_trazabilidad}")
cerrar_conexion(connection)

Conexion Exitosa
conexion cerrada


In [6]:
tbl_pedidos_pendiente = pd.DataFrame(pedidospendiente)
tbl_trazabalidad_Compras = pd.DataFrame(trazabilidad)

In [7]:
tbl_pedidos_pendiente['identificador'] = tbl_pedidos_pendiente['doc_compra'] + tbl_pedidos_pendiente['pos']
tbl_trazabalidad_Compras['identificador'] = tbl_trazabalidad_Compras['Doc_compra'] + tbl_trazabalidad_Compras['Pos_doc_cmp']

In [8]:
pedidos_trazabilidad= tbl_pedidos_pendiente.merge(tbl_trazabalidad_Compras,how='left',on='identificador')

In [9]:
pedidos_trazabilidad.drop(columns='Doc_compra',inplace=True)
pedidos_trazabilidad.drop(columns='Pos_doc_cmp',inplace=True)
pedidos_trazabilidad.drop(columns='id',inplace=True)
pedidos_trazabilidad.drop(columns='fechacarga',inplace=True)

In [36]:
filtrar_pedidosxtrazabilidad = pedidos_trazabilidad.loc[pedidos_trazabilidad['Estado'].isin(['APROBADO','PENDIENTE DE APROBACIÓN'])]

In [ ]:
filtrar_pedidosxtrazabilidad.loc[:, 'Hora'] = pd.to_timedelta(filtrar_pedidosxtrazabilidad['Hora']).apply(lambda x: str(x).split()[2])

In [ ]:
connection = crear_conexion()
insert_data(connection,rpt_estado_compras_pendientes,filtrar_pedidosxtrazabilidad,cabezeras_estado_compra,batch_size=1000)

insert_plan_pedidospendiente_resumen = f"insert into  {plan_pedidospendientes_resumen} select id,doc_compra,pos,material,centro,por_entr_ctd,fecha_entrega,pedido,num_necesidad,nombre_proveedor from {rpt_estado_compras_pendientes} where por_entr_ctd > 0"

consult_data(connection,insert_plan_pedidospendiente_resumen)
cerrar_conexion(connection)